In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

In [4]:
plt.rcParams['figure.figsize'] = (15, 8)

In [5]:
use_db = pd.read_csv("output/db_obs_preproc.csv").drop(columns="Unnamed: 0")

## Naive Classification
Considering a threshold of 1mg/L of mortality

In [6]:
use_db

,test_cas,exposure_type,obs_duration_mean,conc1_type,conc1_mean,class,tax_order,family,genus,species
0,10108642,S,0.032948,T,46.900000,Actinopterygii,Cypriniformes,Cyprinidae,Carassius,auratus
1,88302,S,-2.148755,A,2.920000,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss
2,1397940,S,0.798587,A,0.000063,Actinopterygii,Perciformes,Centrarchidae,Micropterus,salmoides
3,25474413,S,0.798587,A,0.012000,Actinopterygii,Perciformes,Centrarchidae,Lepomis,macrochirus
4,540727,F,-0.281992,T,144.000000,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss
...,...,...,...,...,...,...,...,...,...,...
56624,7646857,F,-0.721540,T,3.870000,Actinopterygii,Siluriformes,Clariidae,Clarias,gariepinus
56625,333415,AQUA,0.032948,F,1.800000,Actinopterygii,Cypriniformes,Cyprinidae,Cyprinus,carpio
56626,52645531,S,0.798587,A,0.007200,Actinopterygii,Perciformes,Centrarchidae,Lepomis,macrochirus
56627,2545600,R,-0.721540,A,17.500000,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,kisutch


In [16]:
X = use_db.copy()

Setting threshold

In [17]:
y = use_db[["conc1_mean"]].copy().values
y = np.where(y > 1, 1, 0)

Checking if the splitting is balanced:

In [18]:
print(len(y[y==0]), len(y[y==1]))

23412 33217


### Our dumb classifier is the one who always predicts the most frequent class

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X, y, test_size=0.33, random_state=42)

In [21]:
y_dumb = np.ones(len(y_test))
accuracy_score(y_test, y_dumb)

0.585134845890411

### Our dumb regressor is the one who always predicts the median

In [22]:
y = use_db[["conc1_mean"]].copy().values

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X, y, test_size=0.33, random_state=42)

In [26]:
median = np.median(y_test)
y_dumb = np.ones(len(y_test))
mean_squared_error(y_test, y_dumb)

55961839.9764045

As expected, **the loss is huge**